<a href="https://colab.research.google.com/github/7pupuwen/PCB_QDETECTION/blob/main/PCB_Detection_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download -d norbertelter/pcb-defect-dataset -q
!unzip -q pcb-defect-dataset.zip -d pcb_data

Dataset URL: https://www.kaggle.com/datasets/norbertelter/pcb-defect-dataset
License(s): unknown


In [2]:
! pip install pennylane -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 77.1 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd

# Set the label files
train_labels_path = "/content/pcb_data/pcb-defect-dataset/train/labels"
val_labels_path = "/content/pcb_data/pcb-defect-dataset/val/labels"
test_labels_path = "/content/pcb_data/pcb-defect-dataset/test/labels"

# Definition of convert label to csv
def convert_labels_to_csv(labels_path, output_csv):
    data = []

    # read every .txt file
    for file in os.listdir(labels_path):
        if file.endswith(".txt"):
            file_path = os.path.join(labels_path, file)
            image_name = file.replace(".txt", ".jpg")  # The file name corresponding to the image
            with open(file_path, "r") as f:
                lines = f.readlines()  #A .txt may have multiple lines of tags

                for line in lines:
                    values = line.strip().split()
                    class_id = int(values[0])  # Defect category
                    x_center, y_center, width, height = map(float, values[1:])  # Analyze numerical values

                    # Save to DataFrame
                    data.append([image_name, class_id, x_center, y_center, width, height])

    # create DataFrame
    df = pd.DataFrame(data, columns=["image_name", "class_id", "x_center", "y_center", "width", "height"])

    # Save as CSV
    df.to_csv(output_csv, index=False)

    print(f"{output_csv} convert finish！")
    print(df.head(10))  # show the first 10 data

# covert `train`、`val`、`test` labels
convert_labels_to_csv(train_labels_path, "train_labels.csv")
convert_labels_to_csv(val_labels_path, "val_labels.csv")
convert_labels_to_csv(test_labels_path, "test_labels.csv")

train_labels.csv convert finish！
                                     image_name  class_id  x_center  y_center  \
0       rotation_90_light_05_short_10_1_600.jpg         3    0.3317    0.0992   
1            l_light_08_mouse_bite_09_4_600.jpg         0    0.4958    0.1808   
2            l_light_08_mouse_bite_09_4_600.jpg         0    0.8617    0.6375   
3  rotation_90_light_11_mouse_bite_03_2_600.jpg         0    0.4517    0.1783   
4  rotation_90_light_11_mouse_bite_03_2_600.jpg         0    0.4433    0.5958   
5  rotation_90_light_11_mouse_bite_03_2_600.jpg         0    0.2100    0.5217   
6                 l_light_09_short_10_3_600.jpg         3    0.5092    0.0333   
7                 l_light_09_short_10_3_600.jpg         3    0.9233    0.4333   
8         light_11_spurious_copper_03_5_256.jpg         5    0.1592    0.1408   
9         light_11_spurious_copper_03_5_256.jpg         5    0.7342    0.5558   

    width  height  
0  0.0567  0.0717  
1  0.0383  0.0417  
2  0.0400  0.04

In [4]:
import os
import cv2
import numpy as np
import pandas as pd

#Set up image folder
train_images_path = "/content/pcb_data/pcb-defect-dataset/train/images"
val_images_path = "/content/pcb_data/pcb-defect-dataset/val/images"
test_images_path = "/content/pcb_data/pcb-defect-dataset/test/images"

# Read tag CSV
train_labels_csv = "train_labels.csv"
val_labels_csv = "val_labels.csv"
test_labels_csv = "test_labels.csv"

# Define functions to read images and match tags
def load_images_and_labels(images_path, labels_csv):
    df = pd.read_csv(labels_csv)  # Read CSV
    X, y = [], []  # Store images and labels

    # Read images and match tags
    for _, row in df.iterrows():
        img_path = os.path.join(images_path, row["image_name"])

        if os.path.exists(img_path):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
            img = cv2.resize(img, (64, 64)) / 255.0  # standardization
            X.append(img)
            y.append(row["class_id"])  # Take only defect categories as labels

    # NumPy array
    X = np.array(X)
    y = np.array(y)

    print(f"{labels_csv} processed, image data size: {X.shape}, label data size: {y.shape}")
    return X, y

# read `train`、`val`、`test` data
X_train, y_train = load_images_and_labels(train_images_path, train_labels_csv)
X_val, y_val = load_images_and_labels(val_images_path, val_labels_csv)
X_test, y_test = load_images_and_labels(test_images_path, test_labels_csv)

train_labels.csv processed, image data size: (12991, 64, 64), label data size: (12991,)
val_labels.csv processed, image data size: (1595, 64, 64), label data size: (1595,)
test_labels.csv processed, image data size: (1662, 64, 64), label data size: (1662,)


In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

# take 500 data
X_train, _, y_train, _ = train_test_split(X_train, y_train, train_size=500, random_state=42, stratify=y_train)
X_val, _, y_val, _ = train_test_split(X_val, y_val, train_size=500, random_state=42, stratify=y_val)
X_test, _, y_test, _ = train_test_split(X_test, y_test, train_size=500, random_state=42, stratify=y_test)

# Confirm data shape
print("取樣後的影像數據 (Train):", X_train.shape)  # (500, 64, 64)
print("取樣後的標籤數據 (Train):", y_train.shape)  # (500,)

print("取樣後的影像數據 (Val):", X_val.shape)  # (500, 64, 64)
print("取樣後的標籤數據 (Val):", y_val.shape)  # (500,)

print("取樣後的影像數據 (Test):", X_test.shape)  # (500, 64, 64)
print("取樣後的標籤數據 (Test):", y_test.shape)  # (500,)


取樣後的影像數據 (Train): (500, 64, 64)
取樣後的標籤數據 (Train): (500,)
取樣後的影像數據 (Val): (500, 64, 64)
取樣後的標籤數據 (Val): (500,)
取樣後的影像數據 (Test): (500, 64, 64)
取樣後的標籤數據 (Test): (500,)


In [6]:
import pennylane as qml

# Set up the quantum computer simulator (4 qubits, corresponding to 2x2 patch)
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def quantum_kernel(x):
    """
    This is a quantum circuit that converts image patches into quantum features
    """
    for i in range(n_qubits):
        qml.RY(x[i] * np.pi, wires=i)  # Embedding image data using Ry gates
        qml.Hadamard(wires=i)  # Join Hadamard Gate

    # oin Quantum Entanglement
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i+1])

    return qml.expval(qml.PauliZ(0))  # Output quantum features

In [7]:
def quanvolution(image):
    output_size = (image.shape[0] //2, image.shape[1] //2)
    q_image = np.zeros(output_size)

    for i in range(0, image.shape[0], 2):
        for j in range(0, image.shape[1], 2):
          patch = image[i:i+2, j:j+2].flatten()
          q_image[i//2, j//2] = quantum_kernel(patch)

    return q_image.flatten()

In [8]:
# Do Quanvolution to data
X_train_quanv = np.array([quanvolution(img) for img in X_train])
X_val_quanv = np.array([quanvolution(img) for img in X_val])
X_test_quanv = np.array([quanvolution(img) for img in X_test])

print("Quantum feature size（Train）:", X_train_quanv.shape)
print("Quantum feature size（Val）:", X_val_quanv.shape)
print("Quantum feature size（Test）:", X_test_quanv.shape)


Quantum feature size（Train）: (500, 1024)
Quantum feature size（Val）: (500, 1024)
Quantum feature size（Test）: (500, 1024)


In [9]:
# import numpy as np

# def compute_kernel_matrix(X):
#     """
#     Compute Quantum Kernel Matrix
#     - X: quantum eigenvector (N, d), where N is the number of samples and d is the number of features
#     - returns the Kernel matrix of (N, N)
#     """
#     N = len(X)
#     kernel_matrix = np.zeros((N, N))

#     for i in range(N):
#         for j in range(i, N):  # 只計算上三角矩陣（因為對稱）
#             kernel_matrix[i, j] = np.dot(X[i], X[j])  # 內積當作 Kernel
#             kernel_matrix[j, i] = kernel_matrix[i, j]  # 利用對稱性

#     return kernel_matrix

In [13]:
def compute_kernel_matrix(X, gamma=0.5):
    N = len(X)
    kernel_matrix = np.zeros((N, N))

    for i in range(N):
        for j in range(i, N):
            dist = np.linalg.norm(X[i] - X[j]) ** 2
            kernel_matrix[i, j] = np.exp(-gamma * dist)  # Gaussian Kernel
            kernel_matrix[j, i] = kernel_matrix[i, j]

    return kernel_matrix

In [14]:
# Quantum Kernel Matrix
train_kernel_matrix = compute_kernel_matrix(X_train_quanv)
val_kernel_matrix = np.array([[np.dot(x1, x2) for x2 in X_train_quanv] for x1 in X_val_quanv])
test_kernel_matrix = np.array([[np.dot(x1, x2) for x2 in X_train_quanv] for x1 in X_test_quanv])

print("Train Kernel Matrix:", train_kernel_matrix.shape)  # (N_train, N_train)
print("Val Kernel Matrix:", val_kernel_matrix.shape)      # (N_val, N_train)
print("Test Kernel Matrix:", test_kernel_matrix.shape)    # (N_test, N_train)

Train Kernel Matrix: (500, 500)
Val Kernel Matrix: (500, 500)
Test Kernel Matrix: (500, 500)


In [15]:
from sklearn import svm
from sklearn.metrics import accuracy_score

# train Quantum SVM
qsvm = svm.SVC(kernel="precomputed")
qsvm.fit(train_kernel_matrix, y_train)


SVC(kernel='precomputed')

In [16]:
# test  QSVM in val set
preds_val = qsvm.predict(val_kernel_matrix)
accuracy_val = accuracy_score(y_val, preds_val)
print(f"QSVM Validation Accuracy: {accuracy_val:.4f}")

# test QSVM in test set
preds_test = qsvm.predict(test_kernel_matrix)
accuracy_test = accuracy_score(y_test, preds_test)
print(f"QSVM Test Accuracy: {accuracy_test:.4f}")


QSVM Validation Accuracy: 0.1620
QSVM Test Accuracy: 0.1600


In [23]:
# Choose label=0&1
X_train_0 = X_train[y_train == 0]
y_train_0 = y_train[y_train == 0]

X_train_1 = X_train[y_train == 1]
y_train_1 = y_train[y_train == 1]

X_val_0 = X_val[y_val == 0]
y_val_0 = y_val[y_val == 0]

X_val_1 = X_val[y_val == 1]
y_val_1 = y_val[y_val == 1]

X_test_0 = X_test[y_test == 0]
y_test_0 = y_test[y_test == 0]

X_test_1 = X_test[y_test == 1]
y_test_1 = y_test[y_test == 1]

print(f"label=0 訓練數據數量: {X_train_0.shape}")
print(f"label=0 訓練數據數量: {X_val_0.shape}")
print(f"label=0 訓練數據數量: {X_test_0.shape}")

print(f"訓練資料 label=1 數量: {X_train_1.shape}")
print(f"驗證資料 label=1 數量: {X_val_1.shape}")
print(f"測試資料 label=1 數量: {X_test_1.shape}")


label=0 訓練數據數量: (85, 64, 64)
label=0 訓練數據數量: (88, 64, 64)
label=0 訓練數據數量: (79, 64, 64)
訓練資料 label=1 數量: (84, 64, 64)
驗證資料 label=1 數量: (82, 64, 64)
測試資料 label=1 數量: (84, 64, 64)


In [24]:
X_train_0 = np.array([quanvolution(img) for img in X_train_0])
X_val_0 = np.array([quanvolution(img) for img in X_val_0])
X_test_0 = np.array([quanvolution(img) for img in X_test_0])

print(" X_train_0 shape:", X_train_0.shape)  # (84, 特徵數)
print(" X_val_0 shape:", X_val_0.shape)      # (88, 特徵數)
print("X_test_0 shape:", X_test_0.shape)    # (79, 特徵數)

 X_train_1 shape: (85, 1024)
 X_val_1 shape: (88, 1024)
X_test_1 shape: (79, 1024)


In [29]:
X_train_1 = np.array([quanvolution(img) for img in X_train_1])
X_val_1 = np.array([quanvolution(img) for img in X_val_1])
X_test_1 = np.array([quanvolution(img) for img in X_test_1])

print(" X_train_1 shape:", X_train_1.shape)  # (84, 特徵數)
print(" X_val_1 shape:", X_val_1.shape)      # (82, 特徵數)
print("X_test_1 shape:", X_test_1.shape)    # (84, 特徵數)

 X_train_1 shape: (84, 1024)
 X_val_1 shape: (82, 1024)
X_test_1 shape: (84, 1024)


In [31]:
# 合併 label=0 和 label=1 的量子特徵數據
X_train_01 = np.vstack([X_train_0, X_train_1])
y_train_01 = np.hstack([y_train_0, y_train_1])

print(f"合併後的訓練數據: {X_train_01.shape}")
print(f"合併後的標籤數據: {y_train_01.shape}")

X_val_01 = np.vstack([X_val_0, X_val_1])
y_val_01 = np.hstack([y_val_0, y_val_1])

print(f"合併後的訓練數據: {X_val_01.shape}")
print(f"合併後的標籤數據: {y_val_01.shape}")

X_test_01 = np.vstack([X_test_0, X_test_1])
y_test_01 = np.hstack([y_test_0, y_test_1])

print(f"合併後的訓練數據: {X_test_01.shape}")
print(f"合併後的標籤數據: {y_test_01.shape}")

合併後的訓練數據: (169, 1024)
合併後的標籤數據: (169,)
合併後的訓練數據: (170, 1024)
合併後的標籤數據: (170,)
合併後的訓練數據: (163, 1024)
合併後的標籤數據: (163,)


In [33]:
# 計算 Kernel Matrix（只使用 label=1 的數據）
train_kernel_matrix_01 = compute_kernel_matrix(X_train_01)
val_kernel_matrix_01 = np.array([[np.dot(x1, x2) for x2 in X_train_01] for x1 in X_val_01])
test_kernel_matrix_01 = np.array([[np.dot(x1, x2) for x2 in X_train_01] for x1 in X_test_01])

print(f"Train Kernel Matrix (label=1): {train_kernel_matrix_1.shape}")
print(f"Val Kernel Matrix (label=1): {val_kernel_matrix_1.shape}")
print(f"Test Kernel Matrix (label=1): {test_kernel_matrix_1.shape}")

Train Kernel Matrix (label=1): (169, 169)
Val Kernel Matrix (label=1): (170, 169)
Test Kernel Matrix (label=1): (163, 169)


In [35]:
from sklearn import svm
from sklearn.metrics import accuracy_score

# 訓練 Quantum SVM
qsvm = svm.SVC(kernel="precomputed")
qsvm.fit(train_kernel_matrix_01, y_train_01)

# 測試 QSVM 在驗證集
preds_val_1 = qsvm.predict(val_kernel_matrix_01)
accuracy_val_1 = accuracy_score(y_val_01, preds_val_1)
print(f"QSVM Validation Accuracy (label=1): {accuracy_val_1:.4f}")

# 測試 QSVM 在測試集
preds_test_1 = qsvm.predict(test_kernel_matrix_01)
accuracy_test_1 = accuracy_score(y_test_01, preds_test_1)
print(f"QSVM Test Accuracy (label=1): {accuracy_test_1:.4f}")

QSVM Validation Accuracy (label=1): 0.5176
QSVM Test Accuracy (label=1): 0.4847


In [36]:
# 傳統 SVM（使用 RBF 核）
classical_svm = svm.SVC(kernel="rbf")
classical_svm.fit(X_train_01, y_train_01)
preds_classical = classical_svm.predict(X_test_01)
acc_classical = accuracy_score(y_test_01, preds_classical)

print(f"Classical SVM Accuracy: {acc_classical:.4f}")

Classical SVM Accuracy: 0.4601
